In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [2]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + [y[1], ] +  y[2].split(',')[2:]

In [3]:
def mapreduce_join_util(x):
#     print x[0], x[1]
    first_d = []
    second_d = []
    for u, v in x[1]:
        if u == 1:
            first_d.append(v)
        else:
            second_d.append(v)
    return [(u, v) for u in first_d for v in second_d]

In [4]:
def mapreduce_join(rdd1, rdd2):
    rdd1_with_record_type = rdd1.map(lambda x: (x[0], [(1, x[1])]))
    rdd2_with_record_type = rdd2.map(lambda x: (x[0],[(2, x[1])]))
    combined_rdd = rdd1_with_record_type.union(rdd2_with_record_type).reduceByKey(lambda x,y : x+y)
    return combined_rdd.flatMap(mapreduce_join_util)

In [5]:
df = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')
df_movies_cleaned = df_movies.map(clean_df_movies)
df_movies_cleaned.take(5)
movieid_moviename = df_movies_cleaned.map(lambda x : (str(x[0]), str(x[1])))

In [6]:
column_data = df.map(lambda x: x.split(","))

In [7]:
movieid_rating = column_data.map(lambda x:(str(x[1]),(float(x[2]), 1)))

In [8]:
f = movieid_rating.reduceByKey(lambda x, y: (x[0]+y[0],x[1]+y[1]))

In [9]:
movie_avgrating = f.map(lambda x:(x[0],x[1][0]/x[1][1]))

In [10]:
moviename_avgrating = mapreduce_join(movieid_moviename, movie_avgrating)

In [11]:
moviename_avgrating.takeOrdered(10, key = lambda x: -x[1])

[('Great Day in Harlem, A (1994)', 5.0),
 ('Letter From Death Row, A (1998)', 5.0),
 ('Marlene Dietrich: Shadow and Light (1996) ', 5.0),
 ('Tough and Deadly (1995)', 5.0),
 ('Entertaining Angels: The Dorothy Day Story (1996)', 5.0),
 ('Aiqing wansui (1994)', 5.0),
 ('Star Kid (1997)', 5.0),
 ('They Made Me a Criminal (1939)', 5.0),
 ('Santa with Muscles (1996)', 5.0),
 ('Saint of Fort Washington, The (1993)', 5.0)]